# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window

In [2]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
spark = (
    SparkSession
    .builder
    .appName("wrangling")
    .getOrCreate()
)

# 3) read in the data set located at the path "data/sparkify_log_small.json"
data = spark.read.load("data/sparkify_log_small.json", format="json")

# 4) create a view to use with your SQL queries
data.createOrReplaceTempView("userLog")

# 5) write code to answer the quiz questions 

# Question 1

Which page did user id ""(empty string) NOT visit?

In [4]:
# TODO: write your code to answer question 1
spark.sql("""
    SELECT
        DISTINCT page
    FROM
        userLog
    WHERE
        page NOT IN (
            SELECT
                DISTINCT page
            FROM
                userLog
            WHERE
                userId = ''
        )
""").show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [6]:
# TODO: write your code to answer question 3
spark.sql("""
    SELECT
        COUNT(DISTINCT userId)
    FROM
        userLog
    WHERE
        gender = "F"
""").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [10]:
# TODO: write your code to answer question 4
spark.sql("""
    SELECT
        artist,
        COUNT(page)
    FROM
        userLog
    WHERE
        page = "NextSong"
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1
""").show()

+--------+-----------+
|  artist|count(page)|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [24]:
spark.sql("""
    WITH isHome AS (
        SELECT
            userId,
            page,
            ts,
            CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home
        FROM
            userLog
        WHERE
            page IN ('NextSong', 'Home')
    ),
    
    cusum AS (
        SELECT
            *,
            SUM(is_home) OVER (PARTITION BY userId ORDER BY ts DESC) AS period
        FROM
            isHome
    ),
    
    counts AS (
        SELECT
            userId,
            period,
            COUNT(*) AS results
        FROM
            cusum
        WHERE
            page = 'NextSong'
        GROUP BY 1, 2
    )
    
    SELECT
        AVG(results)
    FROM 
        counts
""").show()

+-----------------+
|     avg(results)|
+-----------------+
|6.898347107438017|
+-----------------+

